<a href="https://colab.research.google.com/github/evanspap/colab/blob/main/number_theory/Carmichael_prime_factorization_optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prime Factorization — Step by Step (Trial Division)

This notebook provides an **educational, traceable** implementation of prime factorization.

It includes:

- A **step-by-step** factorization routine (verbose tracing)
- A **prime-only candidate** optimization (build primes incrementally; test only primes as divisors)
- Sorted/pretty output formatting
- A simple logical flow diagram

> Notes:
> - Python `int` supports **arbitrary precision** (no overflow like `numpy.int64`).
> - Trial division is not optimal for very large integers (e.g., 20+ digits), but is excellent for learning and small/medium inputs.


## Logical Flow Diagram

```
START
 │
 │  n given
 │
 ├── Handle factor 2 separately
 │
 ├── p = 3
 │
 ├── WHILE p² ≤ n
 │      │
 │      ├── (prime-only version) accept p only if p is prime
 │      │
 │      ├── IF n % p == 0
 │      │      │
 │      │      ├── WHILE n % p == 0
 │      │      │      ├── count p
 │      │      │      ├── n = n / p
 │      │      │      └── record factor p
 │      │      │
 │      │      └── p fully removed
 │      │
 │      └── p = p + 2
 │
 ├── IF n > 1
 │      └── n is prime → record it
 │
 └── END
```


In [22]:
def pretty_factorization(n_original: int, factors: dict) -> str:
    """Return a human-readable factorization string: n = 2^a × 3^b × ..."""
    parts = []
    for p in sorted(factors):
        exp = factors[p]
        parts.append(f"{p}" if exp == 1 else f"{p}^{exp}")
    return f"{n_original} = " + " × ".join(parts)


In [23]:
def is_prime_by_primes(x: int, primes: list[int]) -> bool:
    """
    Check if x is prime using ONLY the primes already found.
    Tests divisibility by primes q up to q*q <= x.
    """
    for q in primes:
        if q * q > x:
            break
        if x % q == 0:
            return False
    return True


In [24]:
def factorize_prime_only(n: int, verbose: bool = False):
    """
    Factorization by trial division using only primes as candidate divisors.
    Builds a prime list incrementally.

    Returns:
      (original_n, factors_dict, primes_tested, found_primes, prime_list_built)
    """
    if n <= 1:
        raise ValueError("n must be > 1")

    original_n = n
    factors = {}          # {prime: exponent}
    primes_tested = []    # primes that were actually used as candidate divisors
    found_primes = []     # prime factors found (with multiplicity, in extraction order)

    # Confirmed primes used for primality testing of candidates
    prime_list = [2]

    # --- Handle factor 2 separately ---
    if verbose:
        print("➡ Checking factor 2")

    while n % 2 == 0:
        if verbose:
            print(f"  {n} ÷ 2 = {n//2}")
        factors[2] = factors.get(2, 0) + 1
        n //= 2
        found_primes.append(2)

    if verbose:
        print(f"  → Reduced n = {n}")
        print("-" * 60)

    # --- Now test only prime odd candidates (3, 5, 7, 11, 13, ...) ---
    p = 3
    if verbose:
        print("➡ Checking only prime divisors up to √n (prime-only candidates)\n")

    # Stop when p^2 > n (no smaller divisor can remain)
    while p * p <= n:

        # Decide if candidate p is prime, using the primes already known.
        # If p is composite, skip it (do not use it as a divisor for n).
        if is_prime_by_primes(p, prime_list):
            prime_list.append(p)
            primes_tested.append(p)

            if verbose:
                print(f"✅ Candidate divisor accepted (prime): p = {p}")

            if n % p == 0:
                if verbose:
                    print(f"✔ {p} divides n")

                # Remove all copies of p from n, counting the exponent
                while n % p == 0:
                    if verbose:
                        print(f"  {n} ÷ {p} = {n//p}")
                    factors[p] = factors.get(p, 0) + 1
                    n //= p
                    found_primes.append(p)

                if verbose:
                    print(f"  → Total power: {p}^{factors[p]}")
                    print(f"  → Reduced n = {n}\n")
            else:
                if verbose:
                    print(f"✘ {p} does not divide n\n")

        #else:
            #if verbose:
                #print(f"⏭ Skipping composite candidate p = {p}")

        p += 2

    # If anything remains > 1, it must be prime.
    if n > 1:
        if verbose:
            print(f"➡ Remaining number {n} is prime; recording it.")
        factors[n] = factors.get(n, 0) + 1
        found_primes.append(n)

    return original_n, factors, primes_tested, found_primes, prime_list

## Run

Enter an integer `n > 1`.  
The notebook will trace each step and then print a sorted, “pretty” final result.


In [25]:
n = int(input("Enter an integer n (>1): "))

Enter an integer n (>1): 12345678901234567890


In [26]:
original_n, factors, primes_tested, found_primes, prime_list = factorize_prime_only(n, verbose=False)
print("-" * 60)

#print("Factors (sorted):")
#for p in sorted(factors):
#    print(f"  {p}^{factors[p]}")

print("\nPrime factorization:")
print(factors)

print("\nPretty form:")
print(pretty_factorization(original_n, factors))

#print("\nPrimes used as candidate divisors (primes_tested):")
#print(primes_tested)

print("\nPrime factors found (with multiplicity) (found_primes):")
print(found_primes)

#print("\nPrime list built during the run (prime_list):")
#print(prime_list)


------------------------------------------------------------

Prime factorization:
{2: 1, 3: 2, 5: 1, 101: 1, 3541: 1, 3607: 1, 3803: 1, 27961: 1}

Pretty form:
12345678901234567890 = 2 × 3^2 × 5 × 101 × 3541 × 3607 × 3803 × 27961

Prime factors found (with multiplicity) (found_primes):
[2, 3, 3, 5, 101, 3541, 3607, 3803, 27961]


### Euler's Totient Function $\varphi(n)$

Once the prime factorization of an integer $n$ is known,

$$n = \prod_i p_i^{a_i},$$

Euler's totient function can be computed using the formula:

$$\varphi(n) = \prod_i p_i^{a_i - 1}(p_i - 1).$$

This follows from the multiplicative property of the totient
function and avoids any need for brute-force counting or
floating-point arithmetic.

The computation below applies this formula directly using the
previously computed prime factorization.



### Euler's Totient Function $\varphi(n)$

If the prime factorization of $n$ is:

$$n = \prod_i p_i^{a_i}$$

then Euler's totient function is given by:

$$\varphi(n) = \prod_i p_i^{a_i - 1}(p_i - 1)$$

This formula follows directly from the multiplicative
property of φ(n) and allows us to compute φ(n) efficiently
once the prime factorization is known.
No floating-point arithmetic is used; all operations
are exact integer computations.

In [27]:
# --- Euler's Totient Function φ(n) using:  φ(n) = ∏ p^(a-1) * (p-1) ---
phi_n = 1
for p, a in factors.items():
    phi_n *= (p ** (a - 1)) * (p - 1)

print("\nEuler's Totient Function:")
print(f"φ({original_n}) = {phi_n}")



Euler's Totient Function:
φ(12345678901234567890) = 3256788124177920000


## Carmichael Function

### Carmichael function — $\lambda(n)$

---

### What is it?

The **Carmichael function** $\lambda(n)$ is an arithmetic function that:

- gives the **exponent of the multiplicative group modulo $n$**,
- is the **smallest positive integer** $k$ such that

$$a^k \equiv 1 \pmod{n} \quad \text{for every } a \text{ with } \gcd(a,n)=1.$$

In words:

> It is the *true minimal universal exponent* that generalizes Euler’s theorem.

---

### Relation to Euler’s Totient Function $\varphi(n)$

It always holds that:

$$\lambda(n) \mid \varphi(n)$$

and typically:

$$\lambda(n) \le \varphi(n).$$

In many cases the inequality is **strict**, which makes  
$\lambda(n)$ a *sharper* bound than $\varphi(n)$.

---

### Formula for $\lambda(n)$ from the prime factorization

If

$$n = \prod_i p_i^{a_i},$$

then:

#### For odd primes $p$:

$$\lambda(p^a) = p^{a-1}(p-1)$$

#### For powers of 2:

$$\lambda(2) = 1,\qquad \lambda(4) = 2,\qquad \lambda(2^a) = 2^{a-2} \quad (a \ge 3)$$

#### General case:

$$\lambda(n) = \text{lcm}\big( \lambda(p_1^{a_1}), \dots, \lambda(p_k^{a_k}) \big)$$

---

### Comparison with Euler’s Totient Formula

Euler’s totient function is:

$$\varphi(n) = \prod_i p_i^{a_i - 1}(p_i - 1)$$

The Carmichael function differs in that it:

- **does not take a product**,
- instead takes the **least common multiple (LCM)** of the same building blocks.


In [28]:
# ------------------------------------------------------------
# Carmichael Function λ(n)
#
# For n = ∏ p^a, the Carmichael function is defined as:
#
#   λ(n) = lcm( λ(p1^a1), λ(p2^a2), ... )
#
# where:
#   - for odd primes p:
#       λ(p^a) = p^(a-1) * (p - 1)
#
#   - for powers of 2:
#       λ(2)   = 1
#       λ(4)   = 2
#       λ(2^a) = 2^(a-2)   for a ≥ 3
#
# This gives the smallest exponent k such that:
#   a^k ≡ 1 (mod n) for all gcd(a, n) = 1
# ------------------------------------------------------------

from math import gcd
from functools import reduce

def lcm(a, b):
    return a * b // gcd(a, b)

lambda_values = []

for p, a in factors.items():
    if p == 2:
        if a == 1:
            lam = 1
        elif a == 2:
            lam = 2
        else:
            lam = 2 ** (a - 2)
    else:
        lam = (p ** (a - 1)) * (p - 1)

    lambda_values.append(lam)

lambda_n = reduce(lcm, lambda_values)

print("\nCarmichael Function:")
print(f"λ({original_n}) = {lambda_n}")



Carmichael Function:
λ(12345678901234567890) = 9423576748200
